In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import plotly.express as px
import scipy.stats as st
import numpy as py



In [2]:
data_df = pd.read_csv('Resources/Crime_Data_from_2020_to_Present.csv')

main_df = data_df[['DR_NO', 'DATE OCC', 
                   'TIME OCC', 'Vict Age', 
                   'Vict Sex','Vict Descent',
                  'LAT', 'LON']]
main_df.count()


KeyboardInterrupt



In [ ]:
# disclaimer for dropping missing age data
cleaning_df = main_df.set_index('Vict Age')
cleaning_df = cleaning_df.drop(index = 0)
cleaning_df = cleaning_df.reset_index()

cleaning_df = cleaning_df.dropna(how='any')


In [ ]:
final_df = cleaning_df.rename(columns={'DR_NO': 'File Number',
                                      'DATE OCC': 'Date Occurred',
                                      'TIME OCC': 'Time Occurred',
                                      'LAT': 'Latitude',
                                      'LON': 'Longitude'})
final_df = final_df.set_index('File Number')
final_df.head()

In [ ]:
# Code to run a heat map for the whole data
fig = px.density_mapbox(final_df, lat = 'Latitude', lon = 'Longitude',
                        radius = 1,
                        center = dict(lat = 34.032, lon = -118.30),
                        zoom = 8.5,
                        range_color = [0,15],
                        mapbox_style = 'open-street-map')
#fig.write_image('Crime_Density.png')
# ^in order to save the heat map figures, the kaleido library needs to be installed on to the conda environment used.
# the code used in the terminal for this is: pip install -U kaleido
fig.show()

In [ ]:
# Code to run heat maps per year
final_df.reset_index()
final_df.groupby(['Year'])
years = set(final_df['Year'])
years_df = final_df.set_index('Year')
years

for year in years:
    df = years_df.loc[year]
    fig = px.density_mapbox(df, lat = 'Latitude', lon = 'Longitude',
                        radius = 1,
                        center = dict(lat = 34.032, lon = -118.30),
                        zoom = 8.5,
                        range_color = [0,10],
                        mapbox_style = 'open-street-map')
    # fig.write_image(f'Crime_Density_{year}.png')
    # ^ again can't be used until kaleido is installed
    fig.show()